In [1]:
import numpy as np
import cv2
from os import listdir

In [4]:
folders = listdir("data")
folders.remove("trainval.txt")
folders.remove("test.txt")
data = np.zeros((3001,96,96))
metadata = np.zeros((3001,6))
metadata_dict = dict()
for f in open("defect_count.csv","r"):
    line = f.split(",",1)
    metadata_dict[line[0]] = line[1][:-1].split(",")
i = 0
for folder in folders:
    sub_folders = listdir("data/{}".format(folder))
    sub_folders = list(filter(lambda x: "not" not in x,sub_folders))
    for sub_folder in sub_folders:
        files = listdir("data/{}/{}".format(folder,sub_folder))
        for file in files:
            img = cv2.imread("data/{}/{}/{}".format(folder,sub_folder,file),0)
            img = cv2.resize(img,(96,96))
            data[i] = img
            test_temp = file.split("_")
            if "temp" in test_temp[-1]:
                metadata[i] = [0,0,0,0,0,0]
            else:
                metadata[i] = metadata_dict[test_temp[0]]                    
            i += 1
    data = data/255.

In [8]:
data = data.reshape((3001,96,96,1))
data[0].shape

In [10]:
metadata[0]

array([1., 2., 1., 1., 0., 0.])

In [11]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input,Dense,Flatten,Conv2D,MaxPool2D,BatchNormalization,Dropout

In [25]:
i = Input(shape = (96,96,1))

x = Conv2D(32,(3,3),activation= "relu")(i)
x = BatchNormalization()(x)
x = MaxPool2D()(x)

x = Conv2D(64,(3,3),activation= "relu")(x)
x = BatchNormalization()(x)
x = MaxPool2D()(x)

x = Conv2D(128,(3,3),activation= "relu")(x)
x = BatchNormalization()(x)
x = MaxPool2D()(x)

x = Flatten()(x)
x = Dense(100,activation = "relu")(x)
x = Dense(100,activation = "relu")(x)
x = Dense(metadata[0].shape[0])(x)

model = Model(i,x)

model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 96, 96, 1)]       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 94, 94, 32)        320       
_________________________________________________________________
batch_normalization_10 (Batc (None, 94, 94, 32)        128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 47, 47, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 45, 45, 64)        18496     
_________________________________________________________________
batch_normalization_11 (Batc (None, 45, 45, 64)        256       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 22, 22, 64)        0   

In [26]:
model.compile(loss = "mse",optimizer = "adam")

In [27]:
r = model.fit(data,metadata,epochs = 5)

Epoch 1/5
94/94 [==============================] - 41s 433ms/step - loss: 2.3809
Epoch 2/5
94/94 [==============================] - 41s 436ms/step - loss: 1.0730
Epoch 3/5
94/94 [==============================] - 41s 435ms/step - loss: 0.7815
Epoch 4/5
94/94 [==============================] - 41s 439ms/step - loss: 0.7606
Epoch 5/5
94/94 [==============================] - 41s 435ms/step - loss: 0.6786
